In [7]:
from XQuant import Tools, Formatter, Config, TradeDate

## Config 常量

In [5]:
print(list(filter(lambda x:'__' not in x, Config.__dict__.keys())))

['database_dir', 'datasets_name', 'datatables', 'stock_table', 'stock_list', 'stock_num_list', 'futures_list', 'trade_date_table', 'trade_date_list', 'quarter_begin', 'quarter_end']


- database_dir: dict, 所有数据的文件夹路径
- datasets_name: list, database_dir的键
- datatables: dict, 具体的表名及相关信息
- stock_table: 股票信息表
- stock_list: 股票编码列表
- stock_num_list: 股票编码（仅数字部分）列表
- futures_list: 期货代号
- trade_date_table: 交易时间表
- trade_date_list: 可交易日期
- quarter_begin: 季度开始日期
- quarter_end: 季度结束日期

## TradeDate 时间管理类

In [9]:
# 判单是否属于日期类字符串/整形
print(TradeDate.is_date("2015-12-16"))
print(TradeDate.is_date("20151216"))
print(TradeDate.is_date(20151216))
print(TradeDate.is_date("2015年12月16日"))

True
True
True
True


In [11]:
# 日期标准化为pd.TimeStamp
TradeDate.format_date("2015年12月16日")

Timestamp('2015-12-16 00:00:00')

In [15]:
# 将区间[begin, end] 进行拓宽, 依据freq将拓展至指定位置
help(TradeDate.extend_date_span)

Help on method extend_date_span in module XQuant.Utils:

extend_date_span(begin: Union[str, int, datetime.datetime, datetime.date, pandas._libs.tslibs.timestamps.Timestamp], end: Union[str, int, datetime.datetime, datetime.date, pandas._libs.tslibs.timestamps.Timestamp], freq: Literal['Q', 'q', 'Y', 'y', 'M', 'm']) -> tuple[datetime.datetime, datetime.datetime] method of builtins.type instance
    将区间[begin, end] 进行拓宽, 依据freq将拓展至指定位置, 详见下
    freq = M :
        [2018-01-04, 2018-04-20] -> [2018-01-01, 2018-04-30]
        [2018-01-01, 2018-04-20] -> [2018-01-01, 2018-04-30]
        [2018-01-04, 2018-04-30] -> [2018-01-01, 2018-04-30]
    freq = Q :
        [2018-01-04, 2018-04-20] -> [2018-01-01, 2018-06-30]
        [2018-01-01, 2018-04-20] -> [2018-01-01, 2018-06-30]
        [2018-01-04, 2018-06-30] -> [2018-01-01, 2018-06-30]
    freq = Y :
        [2018-01-04, 2018-04-20] -> [2018-01-01, 2018-12-31]
        [2018-01-01, 2018-04-20] -> [2018-01-01, 2018-12-31]
        [2018-01-04, 201

In [17]:
# 判断是否属于交易日
TradeDate.is_trade_date("20230811")

True

In [20]:
# 根据lag对交易日进行偏移
print(TradeDate.shift_trade_date("20230811", lag=1))
print(TradeDate.shift_trade_date("20230811", lag=-1))

2023-08-14 00:00:00
2023-08-10 00:00:00


In [21]:
# 返回两个日期之间的所有交易日的列表
print(TradeDate.range_trade_date('20230702', '20230709'))

[Timestamp('2023-07-03 00:00:00'), Timestamp('2023-07-04 00:00:00'), Timestamp('2023-07-05 00:00:00'), Timestamp('2023-07-06 00:00:00'), Timestamp('2023-07-07 00:00:00'), Timestamp('2023-07-10 00:00:00'), Timestamp('2023-07-11 00:00:00')]


In [22]:
# 根据偏移返回其间的交易日的列表
print(TradeDate.range_trade_date('20230702', lag=10))

[Timestamp('2023-07-03 00:00:00'), Timestamp('2023-07-04 00:00:00'), Timestamp('2023-07-05 00:00:00'), Timestamp('2023-07-06 00:00:00'), Timestamp('2023-07-07 00:00:00'), Timestamp('2023-07-10 00:00:00'), Timestamp('2023-07-11 00:00:00'), Timestamp('2023-07-12 00:00:00'), Timestamp('2023-07-13 00:00:00'), Timestamp('2023-07-14 00:00:00'), Timestamp('2023-07-17 00:00:00')]


## Formatter 格式化

is_date, date同TradeDate的is_date, format_date

In [34]:
# 股票期货代码格式化(仅支持单个)
print(Formatter.stock('1'))
print(Formatter.stock(1))
print(Formatter.stock('0001'))
# Formatter.future

000001.SZ
000001.SZ
000001.SZ


In [31]:
# 股票期货代码格式化(支持向量)
Formatter.format_code([1,2,3], kind='stock')

['000001.SZ', '000002.SZ', '000003.SZ']

In [23]:
# 对宽表类型的数据进行格式化
from XQuant.Collector import DataAPI

In [25]:
df = DataAPI.get_data(name='MktEqud', begin='20230101', fields=["ticker", "tradeDate", "marketValue"]).pivot(index='tradeDate', columns='ticker', values='marketValue')
df

ticker,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,900941,900942,900943,900945,900946,900947,900948,900952,900953,900957
tradeDate,,,,,,,,,,,,,,,,,,,,,
2023-01-03,2.672195e+11,2.120278e+11,1.320509e+09,1.979464e+09,8.261969e+09,2.799300e+09,6.562677e+09,3.151799e+10,3.948922e+09,6.770323e+09,...,514960041.0,597361728.6,147987000.0,1.253253e+10,241621070.0,1.317088e+09,4.337591e+09,554634745.5,264320000.0,195091000.0
2023-01-04,2.778927e+11,2.217976e+11,1.323166e+09,1.979464e+09,8.315969e+09,2.726546e+09,6.785141e+09,3.133745e+10,3.959509e+09,6.919317e+09,...,517472041.2,598820487.4,145800000.0,1.257575e+10,240556660.0,1.317088e+09,4.272511e+09,550630162.5,262400000.0,195091000.0
2023-01-05,2.809977e+11,2.248216e+11,1.308553e+09,2.021805e+09,7.991970e+09,2.723081e+09,6.757333e+09,3.162116e+10,3.874814e+09,6.949116e+09,...,519984041.4,601008625.6,144585000.0,1.253253e+10,243749890.0,1.311820e+09,4.321321e+09,554634745.5,288640000.0,203816000.0
2023-01-06,2.837145e+11,2.235422e+11,1.276669e+09,2.000635e+09,7.748971e+09,2.709223e+09,6.701717e+09,3.167275e+10,4.107726e+09,6.812041e+09,...,518728041.3,585691658.2,143127000.0,1.231646e+10,241621070.0,1.306552e+09,4.448228e+09,544623288.0,296960000.0,203118000.0
2023-01-09,2.872076e+11,2.184247e+11,1.287297e+09,1.990049e+09,7.694972e+09,2.757726e+09,6.673909e+09,3.144062e+10,3.938335e+09,6.770323e+09,...,517472041.2,590797314.0,143613000.0,1.248932e+10,245878710.0,1.306552e+09,4.532832e+09,550630162.5,295040000.0,203467000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-24,2.185106e+11,1.646438e+11,2.395251e+09,1.291415e+09,6.250477e+09,1.707989e+09,6.301996e+09,2.955779e+10,2.667905e+09,5.679681e+09,...,537568042.8,545575791.2,122715000.0,8.859205e+09,193100400.0,1.243331e+09,4.532832e+09,438501838.5,251520000.0,172057000.0
2023-07-25,2.274374e+11,1.780062e+11,2.342112e+09,1.323171e+09,6.574476e+09,1.721847e+09,6.410651e+09,2.986730e+10,2.932578e+09,5.900193e+09,...,541336043.1,558704620.4,123444000.0,8.902420e+09,197391520.0,1.259136e+09,4.571880e+09,440504130.0,259200000.0,176943000.0
2023-07-26,2.264671e+11,1.774096e+11,2.575925e+09,1.386683e+09,6.641975e+09,1.721847e+09,6.492142e+09,2.978992e+10,3.017273e+09,6.108786e+09,...,533800042.5,557245861.6,123687000.0,8.945636e+09,194173180.0,1.264405e+09,4.676008e+09,444508713.0,261120000.0,179735000.0


In [26]:
Formatter.dataframe(df)

ticker,600000.SH,600001.SH,600002.SH,600003.SH,600004.SH,600005.SH,600006.SH,600007.SH,600008.SH,600009.SH,...,301387.SZ,301388.SZ,301389.SZ,301391.SZ,301396.SZ,301398.SZ,301408.SZ,301419.SZ,301429.SZ,301439.SZ
2023-01-03,2.122162e+11,NaN,NaN,NaN,3.445942e+10,NaN,1.132000e+10,1.624747e+10,2.114090e+10,1.399771e+11,...,NaN,2.476158e+09,5.406345e+09,6.194120e+09,3.317076e+09,2.526403e+09,NaN,NaN,NaN,NaN
2023-01-04,2.145644e+11,NaN,NaN,NaN,3.460142e+10,NaN,1.132000e+10,1.660002e+10,2.121431e+10,1.404250e+11,...,NaN,2.456693e+09,5.315625e+09,6.232880e+09,3.276869e+09,2.452003e+09,NaN,NaN,NaN,NaN
2023-01-05,2.157385e+11,NaN,NaN,NaN,3.552444e+10,NaN,1.132000e+10,1.656980e+10,2.121431e+10,1.451780e+11,...,NaN,2.482305e+09,5.491395e+09,6.177800e+09,3.290576e+09,2.538403e+09,NaN,NaN,NaN,NaN
2023-01-06,2.154450e+11,NaN,NaN,NaN,3.453042e+10,NaN,1.134000e+10,1.628776e+10,2.106750e+10,1.418434e+11,...,NaN,2.540700e+09,5.446035e+09,6.387240e+09,3.262248e+09,2.550403e+09,NaN,NaN,NaN,NaN
2023-01-09,2.154450e+11,NaN,NaN,NaN,3.609245e+10,NaN,1.144000e+10,1.617696e+10,2.114090e+10,1.456508e+11,...,NaN,2.564263e+09,5.451705e+09,6.738800e+09,3.279610e+09,2.680003e+09,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-24,2.092810e+11,NaN,NaN,NaN,3.225837e+10,NaN,1.186000e+10,1.967223e+10,2.136112e+10,1.089457e+11,...,3.572000e+09,2.689250e+09,5.046300e+09,6.619120e+09,3.947594e+09,5.030400e+09,7.336183e+09,3.983000e+09,2.759255e+09,6.949358e+09
2023-07-25,2.133903e+11,NaN,NaN,NaN,3.318139e+10,NaN,1.240000e+10,2.014565e+10,2.150793e+10,1.113844e+11,...,3.609240e+09,2.721008e+09,5.100165e+09,6.761920e+09,3.985974e+09,5.640000e+09,7.440186e+09,4.035000e+09,2.964958e+09,7.031070e+09
2023-07-26,2.139774e+11,NaN,NaN,NaN,3.282638e+10,NaN,1.194000e+10,2.110257e+10,2.165474e+10,1.095429e+11,...,3.502080e+09,2.657491e+09,5.040630e+09,6.691200e+09,3.875222e+09,5.369600e+09,7.320183e+09,3.795000e+09,3.558422e+09,6.957140e+09
2023-07-27,2.163255e+11,NaN,NaN,NaN,3.289738e+10,NaN,1.204000e+10,2.052842e+10,2.158134e+10,1.105135e+11,...,3.518040e+09,2.755840e+09,4.921560e+09,6.678280e+09,3.750215e+09,5.120000e+09,7.156179e+09,3.738000e+09,3.181300e+09,7.015506e+09


## Tools 工具类

In [35]:
# 数据滞后及清洗
Tools.info_lag(df, n_lag=1, clean=True)

ticker,600000.SH,600001.SH,600002.SH,600003.SH,600004.SH,600005.SH,600006.SH,600007.SH,600008.SH,600009.SH,...,301387.SZ,301388.SZ,301389.SZ,301391.SZ,301396.SZ,301398.SZ,301408.SZ,301419.SZ,301429.SZ,301439.SZ
2023-01-04,2.122162e+11,NaN,NaN,NaN,3.445942e+10,NaN,1.132000e+10,1.624747e+10,2.114090e+10,1.399771e+11,...,NaN,2.476158e+09,5.406345e+09,6.194120e+09,3.317076e+09,2.526403e+09,NaN,NaN,NaN,NaN
2023-01-05,2.145644e+11,NaN,NaN,NaN,3.460142e+10,NaN,1.132000e+10,1.660002e+10,2.121431e+10,1.404250e+11,...,NaN,2.456693e+09,5.315625e+09,6.232880e+09,3.276869e+09,2.452003e+09,NaN,NaN,NaN,NaN
2023-01-06,2.157385e+11,NaN,NaN,NaN,3.552444e+10,NaN,1.132000e+10,1.656980e+10,2.121431e+10,1.451780e+11,...,NaN,2.482305e+09,5.491395e+09,6.177800e+09,3.290576e+09,2.538403e+09,NaN,NaN,NaN,NaN
2023-01-09,2.154450e+11,NaN,NaN,NaN,3.453042e+10,NaN,1.134000e+10,1.628776e+10,2.106750e+10,1.418434e+11,...,NaN,2.540700e+09,5.446035e+09,6.387240e+09,3.262248e+09,2.550403e+09,NaN,NaN,NaN,NaN
2023-01-10,2.154450e+11,NaN,NaN,NaN,3.609245e+10,NaN,1.144000e+10,1.617696e+10,2.114090e+10,1.456508e+11,...,NaN,2.564263e+09,5.451705e+09,6.738800e+09,3.279610e+09,2.680003e+09,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-24,2.089875e+11,NaN,NaN,NaN,3.256604e+10,NaN,1.192000e+10,1.963194e+10,2.128771e+10,1.108618e+11,...,3.587200e+09,2.685152e+09,5.026455e+09,6.703440e+09,3.651525e+09,4.864000e+09,7.360184e+09,3.963000e+09,2.754526e+09,6.926012e+09
2023-07-25,2.092810e+11,NaN,NaN,NaN,3.225837e+10,NaN,1.186000e+10,1.967223e+10,2.136112e+10,1.089457e+11,...,3.572000e+09,2.689250e+09,5.046300e+09,6.619120e+09,3.947594e+09,5.030400e+09,7.336183e+09,3.983000e+09,2.759255e+09,6.949358e+09
2023-07-26,2.133903e+11,NaN,NaN,NaN,3.318139e+10,NaN,1.240000e+10,2.014565e+10,2.150793e+10,1.113844e+11,...,3.609240e+09,2.721008e+09,5.100165e+09,6.761920e+09,3.985974e+09,5.640000e+09,7.440186e+09,4.035000e+09,2.964958e+09,7.031070e+09
2023-07-27,2.139774e+11,NaN,NaN,NaN,3.282638e+10,NaN,1.194000e+10,2.110257e+10,2.165474e+10,1.095429e+11,...,3.502080e+09,2.657491e+09,5.040630e+09,6.691200e+09,3.875222e+09,5.369600e+09,7.320183e+09,3.795000e+09,3.558422e+09,6.957140e+09


Tools.watcher

用作修饰器，监察函数运行时间

In [1]:
from XQuant import Tools
# 对长向量进行批次划分
list(Tools.packaging(series=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], pat=3))

[[0, 1, 2], [3, 4, 5], [6, 7, 8], [9]]

In [9]:
# 默认读取Tokens文件夹中的tokens
Tools.get_config()

{'postgresql': {'host': 'localhost',
  'database': 'QS',
  'user': 'readonly',
  'password': 'read123456',
  'port': '5432'}}

In [2]:
# 获取某个数据表的最新文件路径
Tools.get_newest_file('MktEqud')

'E:\\Share\\Stk_Data\\dataFile\\MktEqud\\MktEqud_Y2023_Q3.h5'

In [3]:
# 根据某个关键词搜索其对应数据表
help(Tools.search_keyword)

Help on method search_keyword in module XQuant.Utils:

search_keyword(keyword: str, fuzzy: bool = True, limit: int = 5, update: bool = False, initial_path: str = WindowsPath('E:/yuankangrui/XQuant/XQuant/Temp/attrs.json'), **kwargs) method of builtins.type instance
    :param initial_path: The initialization path of the log file
    :param keyword: the content you want to search for
    :param fuzzy: fuzzy matching or not
    :param limit: number of the results
    :param update: forced updating
    :return:



In [2]:
Tools.search_keyword("PE", fuzzy=True, limit=3, update=True)

2023-08-12 11:53:01.508 | INFO     | XQuant.Utils:search_keyword:607 - 缓存文件被保存至E:\yuankangrui\XQuant\XQuant\Temp\attrs.json
正在初始化...: 100%|██████████| 440/440 [00:00<00:00, 1198.72it/s, 状态=ResConSecCorederi_sheet 写入成功]                         


{'PE': ['MktEqud', 'MktEqudEval', 'FdmtIndiPSPit'],
 'openPrice': ['MktEqud'],
 'PE1': ['MktEqud', 'MktEqudEval', 'FdmtIndiPSPit']}

In [3]:
help(Tools.varname)

Help on method varname in module XQuant.Utils:

varname(p) method of builtins.type instance
    将变量转化为变量名（字符串）
    :param p: 想要搜寻的变量
    :return: 变量的名字



In [4]:
Tools.varname(Tools)

'Tools'

In [6]:
# 检测主进程使用内存量
Tools.memory_analysis()

Memory Usage: 227.7M
